# Reverse numerical methods applied to channel flow

Our goals here, in general, are to find new valid equations that describe fluid flow. 
We will try random PDEs and use the steady RANS equations to attempt to solve for the Reynolds
stresses in terms of the other quantities we know.


## Steady RANS equations with generic Reynolds stress effects

$$
(\vec{U} \cdot \nabla) \vec{U}
+ \frac{1}{\rho} \nabla P 
- \nu \nabla^2 \vec{U}
= \mathbf{R},
$$

where in this case $\mathbf{R}$ is simply the effects of the Reynolds stresses (i.e., the opposite of the gradient), not the Reynolds stresses themselves.

Now, let's add some terms and assume that their coefficients are isotropic:

$$
(\vec{U} \cdot \nabla) \vec{U}
+ \frac{1}{\rho} \nabla P 
- \nu \nabla^2 \vec{U}
= 
  A (\nabla P)^2
+ B \nabla P^2
+ C (\vec{U} \cdot \nabla) \vec{U}^2
+ D \nabla^2 \vec{U}^2
+ E \nabla K
$$

$$
U \frac{\partial U}{\partial x} 
+ V \frac{\partial U}{\partial y} + W \frac{\partial U}{\partial z}
= - \frac{1}{\rho}\frac{\partial P}{\partial x} + \nu \left( \frac{\partial^2 U}{\partial x^2}
+ \frac{\partial^2 U}{\partial y^2} + \frac{\partial^2 U}{\partial z^2} \right)
+ R_x
$$

$$
U \frac{\partial V}{\partial x} 
+ V \frac{\partial V}{\partial y} + W \frac{\partial V}{\partial z}
= - \frac{1}{\rho}\frac{\partial P}{\partial y} + \nu \left( \frac{\partial^2 V}{\partial x^2}
+ \frac{\partial^2 V}{\partial y^2} + \frac{\partial^2 V}{\partial z^2} \right)
+ R_y
$$

$$
U \frac{\partial W}{\partial x} 
+ V \frac{\partial W}{\partial y} + W \frac{\partial W}{\partial z}
= - \frac{1}{\rho}\frac{\partial P}{\partial z} + \nu \left( \frac{\partial^2 W}{\partial x^2}
+ \frac{\partial^2 W}{\partial y^2} + \frac{\partial^2 W}{\partial z^2} \right)
+ R_z
$$


## Quantities available from JHTDB

* Velocity (`GetVelocity`)
* Pressure (`GetPressure`)
* Velocity gradient (`GetVelocityGradient`)
* Pressure gradient (`GetPressureGradient`)
* Velocity Laplacian (`GetVelocityLaplacian`)
* Velocity Hessian (`GetVelocityHessian`)
* Pressure Hessian (`GetPressureHessian`)


## Algorithm

1. Pick terms (in addition to non-Reynolds stress Navier--Stokes terms).
2. Create a random list of points in space that is at least as large as the number
   of terms.
3. At each point, acquire all data for all terms for all times.
4. Average data at each point for all times.
5. Solve for coefficients using a linear model.

## Terms

$$
U \frac{\partial U}{\partial x} 
+ V \frac{\partial U}{\partial y} + W \frac{\partial U}{\partial z}
+ \frac{1}{\rho}\frac{\partial P}{\partial x} 
- \nu \left( 
    \frac{\partial^2 U}{\partial x^2}
    + \frac{\partial^2 U}{\partial y^2} 
    + \frac{\partial^2 U}{\partial z^2} 
\right)
$$

$$
= 
A \left( \frac{\partial U}{\partial x} \right)^2 
+ B \left( \frac{\partial U}{\partial y} \right)^2
+ C \left( \frac{\partial U}{\partial z} \right)^2
+ D \left( \frac{\partial P}{\partial x} \right)^2
+ E \frac{\partial^2 P}{\partial x^2}
+ F U \frac{\partial P}{\partial x}
$$

$$
U \frac{\partial V}{\partial x} 
+ V \frac{\partial V}{\partial y} 
+ W \frac{\partial V}{\partial z}
+ \frac{1}{\rho}\frac{\partial P}{\partial y} 
- \nu \left( 
    \frac{\partial^2 V}{\partial x^2}
    + \frac{\partial^2 V}{\partial y^2} 
    + \frac{\partial^2 V}{\partial z^2} 
\right)
$$

$$
= 
A \left( \frac{\partial V}{\partial x} \right)^2 
+ B \left( \frac{\partial V}{\partial y} \right)^2
+ C \left( \frac{\partial V}{\partial z} \right)^2
+ D \left( \frac{\partial P}{\partial y} \right)^2
+ E \frac{\partial^2 P}{\partial y^2}
+ F V \frac{\partial P}{\partial y}
$$

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import pyJHTDB
from pyJHTDB import libJHTDB
from pyJHTDB.dbinfo import channel as info

info

In [ ]:
all_times = info["time"]
nu = info["nu"]
all_x = info["xnodes"]
all_y = info["ynodes"]
all_z = info["znodes"]
dataset_name = info["name"]

In [ ]:
db = libJHTDB()
db.initialize()


# Temporarily cut down all_times for testing
all_times = all_times[:3]


def get_data_all_time(x, y, z, f="getVelocity", **kwargs):
    """Get data for all times.
    
    interp_info = [(44, 'FD4Lag4'),
                   (104, 'M1Q4'),
                   (208, 'M2Q8'),
                   (214, 'M2Q14')]
    
    """
    vals = []
    for t in all_times:
        print(f"Getting data at ({x}, {y}, {z}) at t={t}")
        vals.append(db.getData(t, np.array((x, y, z), dtype="float32"), 
                               getFunction=f, sinterp=0, tinterp=0, 
                               data_set=dataset_name, **kwargs))
    return np.asarray(vals)


# Pick 100 random points in space
seed = 5
size = 1
np.random.seed(seed)
x = all_x[np.random.randint(0, high=len(all_x) - 1, size=size)]
y = all_y[np.random.randint(0, high=len(all_y) - 1, size=size)]
z = all_z[np.random.randint(0, high=len(all_z) - 1, size=size)]

# Create a DataFrame for all points and all times
all_data = pd.DataFrame()
for xi, yi, zi in zip(x, y, z):
    df = pd.DataFrame(index=all_times)
    df["x"] = xi
    df["y"] = yi
    df["z"] = zi
    d = get_data_all_time(xi, yi, zi)
    for n in range(d.shape[1]):
        df["col{}".format(n + 1)] = d[:, n]
    all_data = all_data.append(df, ignore_index=True)   


db.finalize()

In [ ]:
all_data.head()